In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os

# WenXuan -- Rare Char Identify Using Selenium

Find the hackmd note for this notebook here: https://hackmd.io/O_egeBmJReKoDQwMQDxPdg



## Char Sinica Function

In [2]:
import re
import urllib
from urllib import request

def char_sinica_search(key, url = 'http://char.iis.sinica.edu.tw/Search/'):
    # search in http://char.iis.sinica.edu.tw/Search/ simply is adding '?SearchValue=' + XXX
    headers = {'User-Agent': 'Mozilla/5.0'}
    URL = urllib.parse.urljoin(url, '?SearchValue=' + urllib.parse.quote(key) )

    # standard way to parse the website
    req  = request.Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    page = request.urlopen(req)
    try:
        soup = BeautifulSoup(page, 'lxml')
    except Exception as e:
        soup = BeautifulSoup(req.text, 'html5lib')

    # get the link to go to the web of the image
    chars = soup.find_all('a', {'target' : "char"})
    if len(chars) != 1:
        print('[Warning] Multiple mathcings. Please change to a specific input.')
        
    for char in chars:
        
        # standard way to parse again
        URL  = urllib.parse.urljoin( url, char['href'] )
        req  = request.Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
        page = request.urlopen(req)
        soup = BeautifulSoup(page, 'lxml')

        # this time we want the unicode
        try:
            _blank = soup.find_all('a', text=re.compile(r' 前往查看 '), attrs={'target':'_blank'})[0]
            UNICODE = _blank['href'].split('/')[-1]

            # and we want the rare_char
            onclick = soup.find_all('input', {'value': '複製Unicode字元', 'type':'button', 'onclick':True})[0]
            rare_char = re.findall(r"addClipboard\('(.)'\)",onclick['onclick'])[0]
            yield UNICODE, rare_char
        except IndexError as e:
            print('[Warning] 查無資料！', e)
            yield None

## Combine JS API and Selenium

In [3]:
text = "樂安任子咸有韜世之量，與余少而歡焉！雖兄弟之愛，無以加也。不幸弱冠而終，良友既沒，何痛如之！其妻又吾姨也，少喪父母，適人而所天又殞，孤女藐焉始孩，斯亦生民之至艱，而荼毒之極哀也。 昔阮瑀既歿，魏文悼之，並命知舊作寡婦之賦。余遂擬之以敘其孤寡之心焉。其辭曰：嗟予生之不造兮，哀天難之匪忱。少伶俜而偏孤兮，痛忉怛以摧心。覽寒泉之遺歎兮，詠蓼莪之餘音。情長慼以永慕兮，思彌遠而逾深。 伊女子之有行兮，爰奉嬪於高族。承慶雲之光覆兮，荷君子之惠渥。顧葛藟之蔓延兮，託微莖於樛木。懼身輕而施重兮，若履冰而臨谷。遵義方之明訓兮，憲女史之典戒。奉蒸嘗以效順兮，供洒掃以彌載。 彼詩人之攸歎兮，徒願言而心痗。何遭命之奇薄兮，遘天禍之未悔。榮華曄其始茂兮，良人忽以捐背。靜闔門以窮居兮，塊煢獨而靡依。易錦茵以苫席兮，代羅幬以素帷。命阿保而就列兮，覽巾箑以舒悲。口嗚咽以失聲兮，淚橫迸而霑衣。 愁煩冤其誰告兮，提孤孩於坐側。 時曖曖而向昏兮，日杳杳而西匿。雀群飛而赴楹兮，雞登棲而斂翼。歸空館而自怜兮，撫衾裯以歎息。思纏緜以瞀亂兮，心摧傷以愴惻。 曜靈曄而遄邁兮，四節運而推移。天凝露以降霜兮，木落葉而隕枝。仰神宇之寥寥兮，瞻靈衣之披披。退幽悲於堂隅兮，進獨拜於牀垂。耳傾想於疇昔兮，目仿佛乎平素。雖冥冥而罔覿兮，猶依依以憑附。痛存亡之殊制兮，將遷神而安厝。龍轜儼其星駕兮，飛旐翩以啟路。輪按軌以徐進兮，馬悲鳴而跼顧。潛靈邈其不反兮，殷憂結而靡訴。睎形影於几筵兮，馳精爽於丘墓。 自仲秋而在疚兮，踰履霜以踐冰。雪霏霏而驟落兮，風瀏瀏而夙興。霤泠泠以夜下兮，水溓溓以微凝。意忽怳以遷越兮，神一夕而九升。庶浸遠而哀降兮，情惻惻而彌甚。願假夢以通靈兮，目烱烱而不寢。夜漫漫以悠悠兮，寒淒淒以凜凜。氣憤薄而乘胸兮，涕交橫而流枕。亡魂逝而永遠兮，時歲忽其遒盡。容貌儡以頓顇兮，左右悽其相慜。感三良之殉秦兮，甘捐生而自引。鞠稚子於懷抱兮，羌低徊而不忍。獨指景而心誓兮，雖形存而志隕。 重曰：仰皇穹兮歎息，私自憐兮何極！省微身兮孤弱，顧稚子兮未識。如涉川兮無梁，若陵虛兮失翼。上瞻兮遺象，下臨兮泉壤。窈冥兮潛翳，心存兮目想。奉虛坐兮肅清，愬空宇兮曠朗。廓孤立兮顧影，塊獨言兮聽響。顧影兮傷摧，聽響兮增哀。遙逝兮逾遠，緬邈兮長乖。 四節流兮忽代序，歲云暮兮日西頹。霜被庭兮風入室，夜既分兮星漢迴。夢良人兮來遊，若閶闔兮洞開。怛驚悟兮無聞，超\uf4eb\uf6a4敞怳兮慟懷。慟懷兮奈何，言陟兮山阿。墓門兮肅肅，脩壟兮峨峨。孤鳥嚶兮悲鳴，長松萋兮振柯。哀鬱結兮交集，淚橫流兮滂沲；蹈恭姜兮明誓，詠柏舟兮清歌。終歸骨兮山足，存憑託兮餘華。要吾君兮同穴，之死矢兮靡佗。"
text

'樂安任子咸有韜世之量，與余少而歡焉！雖兄弟之愛，無以加也。不幸弱冠而終，良友既沒，何痛如之！其妻又吾姨也，少喪父母，適人而所天又殞，孤女藐焉始孩，斯亦生民之至艱，而荼毒之極哀也。 昔阮瑀既歿，魏文悼之，並命知舊作寡婦之賦。余遂擬之以敘其孤寡之心焉。其辭曰：嗟予生之不造兮，哀天難之匪忱。少伶俜而偏孤兮，痛忉怛以摧心。覽寒泉之遺歎兮，詠蓼莪之餘音。情長慼以永慕兮，思彌遠而逾深。 伊女子之有行兮，爰奉嬪於高族。承慶雲之光覆兮，荷君子之惠渥。顧葛藟之蔓延兮，託微莖於樛木。懼身輕而施重兮，若履冰而臨谷。遵義方之明訓兮，憲女史之典戒。奉蒸嘗以效順兮，供洒掃以彌載。 彼詩人之攸歎兮，徒願言而心痗。何遭命之奇薄兮，遘天禍之未悔。榮華曄其始茂兮，良人忽以捐背。靜闔門以窮居兮，塊煢獨而靡依。易錦茵以苫席兮，代羅幬以素帷。命阿保而就列兮，覽巾箑以舒悲。口嗚咽以失聲兮，淚橫迸而霑衣。 愁煩冤其誰告兮，提孤孩於坐側。 時曖曖而向昏兮，日杳杳而西匿。雀群飛而赴楹兮，雞登棲而斂翼。歸空館而自怜兮，撫衾裯以歎息。思纏緜以瞀亂兮，心摧傷以愴惻。 曜靈曄而遄邁兮，四節運而推移。天凝露以降霜兮，木落葉而隕枝。仰神宇之寥寥兮，瞻靈衣之披披。退幽悲於堂隅兮，進獨拜於牀垂。耳傾想於疇昔兮，目仿佛乎平素。雖冥冥而罔覿兮，猶依依以憑附。痛存亡之殊制兮，將遷神而安厝。龍轜儼其星駕兮，飛旐翩以啟路。輪按軌以徐進兮，馬悲鳴而跼顧。潛靈邈其不反兮，殷憂結而靡訴。睎形影於几筵兮，馳精爽於丘墓。 自仲秋而在疚兮，踰履霜以踐冰。雪霏霏而驟落兮，風瀏瀏而夙興。霤泠泠以夜下兮，水溓溓以微凝。意忽怳以遷越兮，神一夕而九升。庶浸遠而哀降兮，情惻惻而彌甚。願假夢以通靈兮，目烱烱而不寢。夜漫漫以悠悠兮，寒淒淒以凜凜。氣憤薄而乘胸兮，涕交橫而流枕。亡魂逝而永遠兮，時歲忽其遒盡。容貌儡以頓顇兮，左右悽其相慜。感三良之殉秦兮，甘捐生而自引。鞠稚子於懷抱兮，羌低徊而不忍。獨指景而心誓兮，雖形存而志隕。 重曰：仰皇穹兮歎息，私自憐兮何極！省微身兮孤弱，顧稚子兮未識。如涉川兮無梁，若陵虛兮失翼。上瞻兮遺象，下臨兮泉壤。窈冥兮潛翳，心存兮目想。奉虛坐兮肅清，愬空宇兮曠朗。廓孤立兮顧影，塊獨言兮聽響。顧影兮傷摧，聽響兮增哀。遙逝兮逾遠，緬邈兮長乖。 四節流兮忽代序，歲云暮兮日西頹。霜被庭兮風入室，夜既分兮星漢迴。夢良人兮來遊，若閶闔兮洞開。怛驚悟兮無聞

In [4]:
# append JS_API to the bottom of the text file
JS_API = """
<script language="javascript">
window.history.forward(1);
</script>
<script src="http://char.iis.sinica.edu.tw/API/ics.js" language="javascript"></script>
<script language="JavaScript">
processPage("red","24", "DFKai-sb");
</script>
"""

with open("temp.html", "w", encoding="utf-8") as file:
    file.write(text + JS_API)

In [5]:
driver_path = '/Users/cicero/Documents/Physics/PYTHON/Scraping/chromedriver'

# open the chrome driver
driver = webdriver.Chrome(executable_path=driver_path)
driver.get("file://" + os.path.join(os.getcwd(), "temp.html"))

# get the page source after running the JS API
page = driver.page_source

# close the diver
driver.close()

# remove the file
os.remove("temp.html")

# convert the page to bs4 object
soup = BeautifulSoup(page, "lxml")

# acquire the bag of rare chars components
bag_of_rare_char = [char["alt"] for char in soup.find_all("img")]

# convert rare char to unicode
bag_of_rare_char_unicode = [
    next(iter(char_sinica_search(char["alt"]))) for char in soup.find_all("img")
]

[Warning] Multiple mathcings. Please change to a specific input.


In [6]:
bag_of_rare_char

['\uf4eb\uf6a4敞']

In [7]:
bag_of_rare_char_unicode

[('U+2288C', '\uf3f2')]